# Model 2's Implementation: Using VGG 19 CNN Layers.

## Intialise and import libraries required

In [ ]:
import os
from random import shuffle
from math import floor
from PIL import Image
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten, Input, UpSampling2D
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, Flatten, Dense, Reshape, MaxPooling2D,  MaxPool2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import plot_model
from tensorflow.keras.backend import int_shape
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.vgg19 import VGG19

## !!!! PLEASE SET YOUR WORKING DIRECTORY !!!!

In [ ]:
#SET UP HERE CHANGE THE VALUE TO YOUR FILE DIRECTORY
#
os.chdir('C:/Users/YOUR_USER/Documents/Colourising_Photos/')

#Setting up path for data
pathColor = os.getcwd()+"/Data/colour/"
pathNb = os.getcwd()+"/Data/black/"

#Set resolution (Current model trained for 256x256)
width, height = (256,256)

## Import VGG 19

In [ ]:
vgg19 = VGG19(include_top=False, weights='imagenet' ,input_tensor=None,  pooling=None, input_shape=(width,height,3), classes=1000)

### Using VGG19, set up our input layers and output layers, and fill the middle with VGG19 layers

In [ ]:
#Create input, take 256x256 photographs, this can be changed at a later date.
inputs = Input(name='Model2_Input', shape=(256,256,1)  )
x = Conv2D(filters=3,kernel_size=3, activation='relu',padding='same')(inputs)

#Input part of the VGG19 model, not all layers.
for lyr in vgg19.layers[:-13]:
    x = lyr(x)
    x.trainable=False

#Perform two transpose conv's for 128x128,128, and 256x256,3 shapes
x = Conv2DTranspose(kernel_size=3,activation='relu',strides = 2 , padding='same',filters=128)(x)
x = Conv2DTranspose(kernel_size=3,activation='relu',strides = 2 , padding='same',filters=3)(x)

#finish the model
model = Model(inputs, x, name='model2')
#Compile using MSE for loss, and adam optimiser
model.compile(loss='mse', optimizer='adam',metrics=['accuracy'])


### Loading data

In [ ]:
#Create holders for training data and its true coloured version
Xtrain = []
Ytrain = []

#Open file and read all photos that are training data
file = open(os.getcwd()+"/Data/train.txt")
fileTrainingData = file.readlines()
file.close()


for f in fileTrainingData:
    Ytrain.append( np.load(pathColor + f[:-1]))
    Xtrain.append( np.load(pathNb + f[:-1]))

# Flatten the data and then reshape it to be (-1,w,h,1)
Xtrain = np.asarray(Xtrain).flatten()
Xtrain = Xtrain.reshape(-1,width,height,1)
Ytrain = np.asarray(Ytrain).flatten()
Ytrain = Ytrain.reshape((-1,width,height,3))
    
#Loading test data
Xtest = []
Ytest = []

file = open(os.getcwd()+"/Data/test.txt")
fileTestData = file.readlines()
file.close()

for f in fileTestData:
    Ytest.append( np.load(pathColor + f[:-1]))
    Xtest.append( np.load(pathNb + f[:-1]))

# Flatten the data and then reshape it to be (-1,w,h,1)
Xtest = np.asarray(Xtest).flatten()
Xtest = Xtest.reshape(-1,width,height,1)
Ytest = np.asarray(Ytest).flatten()
Ytest = Ytest.reshape((-1,width,height,3))

## Training the model

In [ ]:
step = 20
batch = 22
start = 0
end = 1000

for current in range(start,end,step):
    
    
    model.fit(Xtrain, Ytrain, batch_size=batch, validation_data=(Xtest,Ytest), initial_epoch = current, epochs = current+step)
    
    #Create file to dump output to
    os.mkdir(''+os.getcwd()+'/outputs/Model2/Epoch{}'.format(current + step))
    os.mkdir(''+os.getcwd()+'/outputs/Model2/Epoch{}/test'.format(current + step))
    os.mkdir(''+os.getcwd()+'/outputs/Model2/Epoch{}/train'.format(current + step))
 
    # Predict
    output = model.predict(Xtrain[:])
    #Now return the values from 0->1 to 0->255
    output *= 255
    #Convert from f_32 -> u_int8
    output = output.astype('uint8')
    #save train images
    photo_num = 1
    for img in output:
        output = Image.fromarray(img)
        output.save(''+os.getcwd()+'/outputs/Model2/Epoch{}/train/ouput{}.png'.format(current + step,photo_num))
        photo_num += 1
        
    # Predict
    output = model.predict(Xtest[:])
    #Now return the values from 0->1 to 0->255
    output *= 255
    #Convert from f_32 -> u_int8
    output = res.astype('uint8')
    #Save test images
    photo_num = 1
    for img in output:
        output = Image.fromarray(img)
        output.save(''+os.getcwd()+'/outputs/Model2/Epoch{}/test/ouput{}.png'.format(current + step,photo_num))
        photo_num += 1    
        
    #Save model
    model.save( ""+os.getcwd()+"/outputs/Model2/Epoch{}/model2_{}.h5".format(current + step,current + step))